## ensemble 및 cross-validation
## 범주형 데이터

In [1]:
install.packages("caret")
library(caret)
mydata = read.csv("https://stats.idre.ucla.edu/stat/data/binary.csv")
head(mydata)
str(mydata)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘Rcpp’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Loading required package: ggplot2

Loading required package: lattice



,admit,gre,gpa,rank
,<int>,<int>,<dbl>,<int>
1,0,380,3.61,3
2,1,660,3.67,3
3,1,800,4.00,1
4,1,640,3.19,4
5,0,520,2.93,4
6,1,760,3.00,2


'data.frame':	400 obs. of  4 variables:
 $ admit: int  0 1 1 1 0 1 1 0 1 0 ...
 $ gre  : int  380 660 800 640 520 760 560 400 540 700 ...
 $ gpa  : num  3.61 3.67 4 3.19 2.93 3 2.98 3.08 3.39 3.92 ...
 $ rank : int  3 3 1 4 4 2 1 2 3 2 ...


In [2]:
## 변수 factor화
mydata$admit = as.factor(mydata$admit)
mydata$rank = as.factor(mydata$rank)
summary(mydata)

 admit        gre             gpa        rank   
 0:273   Min.   :220.0   Min.   :2.260   1: 61  
 1:127   1st Qu.:520.0   1st Qu.:3.130   2:151  
         Median :580.0   Median :3.395   3:121  
         Mean   :587.7   Mean   :3.390   4: 67  
         3rd Qu.:660.0   3rd Qu.:3.670          
         Max.   :800.0   Max.   :4.000          

In [3]:
## 데이터 분할
set.seed(123)
my.ind = createDataPartition(y = mydata$admit,p = 0.75, list = FALSE)
my.training = mydata[my.ind, ]
my.test = mydata[-my.ind, ]

In [4]:
## bagging & cross-validation
cctrl1 = trainControl(method = "cv", number = 5)
set.seed(123)
model.bag = train(admit ~ ., data = my.training, trControl = cctrl1, method = "bag",
                  tuneGrid = data.frame(vars = seq(5, 20, by = 5)),
                  bagControl = bagControl(fit = ldaBag$fit, predict = ldaBag$pred, aggregate = ldaBag$aggregate))

Warning message:
“executing %dopar% sequentially: no parallel backend registered”


In [5]:
## gradient boosting & cross-validation
install.packages("gbm")
library(gbm)
cctrl1 = trainControl(method = "cv", number = 5)
set.seed(123)
model.gbm = train(admit ~ ., data = my.training, trControl = cctrl1, method = "gbm", tuneLength = 3)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loaded gbm 2.1.8.1



Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        1.2369            -nan     0.1000    0.0054
     2        1.2238            -nan     0.1000    0.0037
     3        1.2163            -nan     0.1000    0.0013
     4        1.2111            -nan     0.1000   -0.0002
     5        1.2028            -nan     0.1000    0.0035
     6        1.1967            -nan     0.1000    0.0027
     7        1.1929            -nan     0.1000    0.0020
     8        1.1877            -nan     0.1000   -0.0009
     9        1.1852            -nan     0.1000    0.0009
    10        1.1821            -nan     0.1000   -0.0026
    20        1.1528            -nan     0.1000   -0.0011
    40        1.1229            -nan     0.1000   -0.0017
    60        1.0993            -nan     0.1000   -0.0006
    80        1.0740            -nan     0.1000   -0.0019
   100        1.0521            -nan     0.1000   -0.0012
   120        1.0388            -nan     0.1000   -0.0026
   140        

In [6]:
## adaboost & cross-validation
install.packages("ada")
library(ada)
cctrl1 = trainControl(method = "cv", number = 5)
set.seed(123)
model.ada = train(admit ~ ., data = my.training, trControl = cctrl1, method = "ada", tuneLength = 3)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: rpart



In [7]:
## random forest & cross-validation
install.packages("randomForest")
library(randomForest)
cctrl1 = trainControl(method = "cv", number = 5)
set.seed(123)
model.rf = train(admit ~ ., data = my.training, trControl = cctrl1, method = "rf", tuneLength = 3)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin




In [8]:
## 예측 및 평가 측도 계산
bag.pred = predict(model.bag, newdata = my.test)
gbm.pred = predict(model.gbm, newdata = my.test)
ada.pred = predict(model.ada, newdata = my.test)
rf.pred = predict(model.rf, newdata = my.test)

In [9]:
confusionMatrix(my.test$admit, bag.pred)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 64  4
         1 24  7
                                          
               Accuracy : 0.7172          
                 95% CI : (0.6178, 0.8031)
    No Information Rate : 0.8889          
    P-Value [Acc > NIR] : 0.9999993       
                                          
                  Kappa : 0.2025          
                                          
 Mcnemar's Test P-Value : 0.0003298       
                                          
            Sensitivity : 0.7273          
            Specificity : 0.6364          
         Pos Pred Value : 0.9412          
         Neg Pred Value : 0.2258          
             Prevalence : 0.8889          
         Detection Rate : 0.6465          
   Detection Prevalence : 0.6869          
      Balanced Accuracy : 0.6818          
                                          
       'Positive' Class : 0               
                                    

In [10]:
confusionMatrix(my.test$admit, gbm.pred)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 62  6
         1 27  4
                                          
               Accuracy : 0.6667          
                 95% CI : (0.5648, 0.7582)
    No Information Rate : 0.899           
    P-Value [Acc > NIR] : 1.0000000       
                                          
                  Kappa : 0.05            
                                          
 Mcnemar's Test P-Value : 0.0004985       
                                          
            Sensitivity : 0.6966          
            Specificity : 0.4000          
         Pos Pred Value : 0.9118          
         Neg Pred Value : 0.1290          
             Prevalence : 0.8990          
         Detection Rate : 0.6263          
   Detection Prevalence : 0.6869          
      Balanced Accuracy : 0.5483          
                                          
       'Positive' Class : 0               
                                    

In [11]:
confusionMatrix(my.test$admit, ada.pred)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 61  7
         1 26  5
                                          
               Accuracy : 0.6667          
                 95% CI : (0.5648, 0.7582)
    No Information Rate : 0.8788          
    P-Value [Acc > NIR] : 1.000000        
                                          
                  Kappa : 0.07            
                                          
 Mcnemar's Test P-Value : 0.001728        
                                          
            Sensitivity : 0.7011          
            Specificity : 0.4167          
         Pos Pred Value : 0.8971          
         Neg Pred Value : 0.1613          
             Prevalence : 0.8788          
         Detection Rate : 0.6162          
   Detection Prevalence : 0.6869          
      Balanced Accuracy : 0.5589          
                                          
       'Positive' Class : 0               
                                    

In [12]:
confusionMatrix(my.test$admit, rf.pred)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 63  5
         1 26  5
                                          
               Accuracy : 0.6869          
                 95% CI : (0.5859, 0.7764)
    No Information Rate : 0.899           
    P-Value [Acc > NIR] : 1.000000        
                                          
                  Kappa : 0.1076          
                                          
 Mcnemar's Test P-Value : 0.000328        
                                          
            Sensitivity : 0.7079          
            Specificity : 0.5000          
         Pos Pred Value : 0.9265          
         Neg Pred Value : 0.1613          
             Prevalence : 0.8990          
         Detection Rate : 0.6364          
   Detection Prevalence : 0.6869          
      Balanced Accuracy : 0.6039          
                                          
       'Positive' Class : 0               
                                    

## 연속형 데이터

In [13]:
## regression 분석을 위한 data 생성
reg.data = read.csv("https://stats.idre.ucla.edu/wp-content/uploads/2019/02/elemapi2v2.csv")
regdata.ind = createDataPartition(y = reg.data$api00, p = 0.75, list = FALSE)
reg.training = reg.data[regdata.ind, ]
reg.test = reg.data[-regdata.ind, ]
summary(reg.training)

      snum           dnum         api00           api99           growth      
 Min.   :  65   Min.   : 41   Min.   :369.0   Min.   :333.0   Min.   :-69.00  
 1st Qu.:1678   1st Qu.:395   1st Qu.:524.0   1st Qu.:484.0   1st Qu.: 19.00  
 Median :3005   Median :401   Median :643.0   Median :605.0   Median : 37.00  
 Mean   :2859   Mean   :453   Mean   :647.9   Mean   :610.5   Mean   : 37.35  
 3rd Qu.:4284   3rd Qu.:630   3rd Qu.:762.0   3rd Qu.:730.0   3rd Qu.: 55.00  
 Max.   :6072   Max.   :796   Max.   :940.0   Max.   :917.0   Max.   :134.00  
                                                                              
     meals             ell            yr_rnd          mobility   
 Min.   :  0.00   Min.   : 0.00   Min.   :0.0000   Min.   : 3.0  
 1st Qu.: 33.00   1st Qu.:10.00   1st Qu.:0.0000   1st Qu.:13.0  
 Median : 67.00   Median :25.00   Median :0.0000   Median :17.0  
 Mean   : 60.31   Mean   :31.01   Mean   :0.2359   Mean   :18.1  
 3rd Qu.: 90.00   3rd Qu.:48.00   3rd 

+ 연속형 데이터에 대한 adaboost는 caret 패키지에서 적용되지 않아 분석 과정에서 제외하였다.

In [14]:
## bagging & cross-validation
install.packages("party")
library(party)
cctrl1 = trainControl(method = "cv", number = 5)
model.bagr = train(api00 ~ enroll + full + col_grad + emer, data = reg.training, trControl = cctrl1,
                   B = 100, method = "bag", tuneGrid = expand.grid(vars = c(10, 15, 20)),
                  bagControl = bagControl(fit = ctreeBag$fit, predict = ctreeBag$pred, aggregate = ctreeBag$aggregate))

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘TH.data’, ‘libcoin’, ‘matrixStats’, ‘multcomp’, ‘mvtnorm’, ‘modeltools’, ‘strucchange’, ‘coin’, ‘zoo’, ‘sandwich’


Loading required package: grid

Loading required package: mvtnorm

Loading required package: modeltools

Loading required package: stats4

Loading required package: strucchange

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: sandwich



In [15]:
## gradient boosting & cross-validation
cctrl1 = trainControl(method = "cv", number = 5)
set.seed(123)
model.gbmr = train(api00 ~ enroll + full + col_grad + emer, data = reg.training,
                   trControl = cctrl1, method = "gbm", tuneLength = 3)

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1    18229.3715            -nan     0.1000 1034.4595
     2    17154.3264            -nan     0.1000 1080.1131
     3    16130.6835            -nan     0.1000  688.9871
     4    15173.7852            -nan     0.1000  810.7302
     5    14431.6362            -nan     0.1000  546.9066
     6    13758.5589            -nan     0.1000  440.3989
     7    13138.7688            -nan     0.1000  575.4356
     8    12707.9588            -nan     0.1000  337.5909
     9    12123.3445            -nan     0.1000  477.7794
    10    11701.5952            -nan     0.1000  387.3108
    20     9020.8942            -nan     0.1000   43.6339
    40     7330.1381            -nan     0.1000  -18.5823
    60     6942.6205            -nan     0.1000  -32.7056
    80     6742.6378            -nan     0.1000   -0.0468
   100     6581.5863            -nan     0.1000  -49.0577
   120     6474.6905            -nan     0.1000   -8.5614
   140     637

In [16]:
## random forest & cross-validation
cctrl1 = trainControl(method = "cv", number = 5)
set.seed(123)
model.rfr = train(api00 ~ enroll + full + col_grad + emer, data = reg.training,
                  trControl = cctrl1, method = "rf", tuneLength = 3)

In [17]:
## 예측 및 평가 측도 계산
bagr.pred = predict(model.bagr, newdata = reg.test)
gbmr.pred = predict(model.gbmr, newdata = reg.test)
rfr.pred = predict(model.rfr, newdata = reg.test)

In [18]:
install.packages("forecast")
library(forecast)
accuracy(ts(bagr.pred), reg.test$api00)
accuracy(ts(gbmr.pred), reg.test$api00)
accuracy(ts(rfr.pred), reg.test$api00)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘xts’, ‘TTR’, ‘quadprog’, ‘quantmod’, ‘fracdiff’, ‘lmtest’, ‘tseries’, ‘urca’, ‘RcppArmadillo’


Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



,ME,RMSE,MAE,MPE,MAPE
Test set,-11.62764,78.38542,61.86196,-3.499486,10.30429


,ME,RMSE,MAE,MPE,MAPE
Test set,-9.176208,78.06324,59.95985,-2.783279,9.950763


,ME,RMSE,MAE,MPE,MAPE
Test set,-11.12786,70.57514,56.20397,-3.203636,9.455154
